In [1]:
import jax
jax.default_backend()

'gpu'

In [2]:
from mujoco import mjx
import mujoco

## Example 1: Single Robot Scene

In [3]:
# Simulate manual scene
from revolve2.simulators.mujoco_simulator._scene_to_model import scene_to_model
from revolve2.simulation.scene import Scene
from revolve2.standards.simulation_parameters import (
    STANDARD_CONTROL_FREQUENCY,
    STANDARD_SIMULATION_TIMESTEP,
)

# Single scene to model
from pyrr import Vector3
from revolve2.experimentation.logging import setup_logging
from revolve2.experimentation.rng import make_rng_time_seed
from revolve2.modular_robot import ModularRobot
from revolve2.modular_robot.body import RightAngles
from revolve2.modular_robot.body.v2 import ActiveHingeV2, BodyV2, BrickV2
from revolve2.modular_robot.brain.cpg import BrainCpgNetworkNeighborRandom
from revolve2.modular_robot_simulation import ModularRobotScene, simulate_scenes
from revolve2.simulation.scene import Pose

from revolve2.simulators.mujoco_simulator import LocalSimulator
from revolve2.standards import terrains
from revolve2.standards.interactive_objects import Ball
from revolve2.standards.simulation_parameters import make_standard_batch_parameters
import logging


In [4]:
def main() -> None:
    """Run the simulation."""
    # Set up logging to give output of your simulation into the command line interface (CLI).
    setup_logging()

    # Set up a random number generator, used later for the brain.
    rng = make_rng_time_seed()

    # Create a body for the robot.
    body = make_body()

    """
    Here we create a brain for the robot.
    We choose a 'CPG' brain with random parameters.
    If you want to know more about CPGs checkout the Methods section in: https://doi.org/10.1038/s41598-023-48338-4. 
    """
    brain = BrainCpgNetworkNeighborRandom(body=body, rng=rng)

    """Once we have a body and a brain we combine it into a ModularRobot."""
    robot = ModularRobot(body, brain)

    """
    To simulate our newly created robot, we create a modular robot scene.
    This scene is a combination of one or more modular robots positioned in a given terrain.
    """
    scene = ModularRobotScene(terrain=terrains.flat())
    scene.add_robot(robot)

    # Additionally to robots you can also add interactive objects to the scene.
    scene.add_interactive_object(
        Ball(radius=0.1, mass=0.1, pose=Pose(Vector3([-0.5, 0.5, 0])))
    )

    """
    After we have the scene ready we create a simulator that will perform the simulation.
    This tutorial chooses to use Mujoco, but your version of revolve might contain other simulators as well.
    
    For mujoco we can select either the `native` mujoco viewer (more performance) or our `custom` viewer (which is more flexible for adjustments).
    """
    simulator = LocalSimulator(viewer_type="native")

    # `batch_parameters` are important parameters for simulation.
    # Here, we use the parameters that are standard in CI Group.
    batch_parameters = make_standard_batch_parameters()
    batch_parameters.simulation_time = 600  # Here we update our simulation time.

    # Simulate the scene.
    # A simulator can run multiple sets of scenes sequentially; it can be reused.
    # However, in this tutorial we only use it once.
    simulate_scenes(
        simulator=simulator,
        batch_parameters=batch_parameters,
        scenes=scene,
    )

main()

[2024-09-29 14:16:30,257] [INFO] [logging] =======================================
[2024-09-29 14:16:30,259] [INFO] [logging] =======================================
[2024-09-29 14:16:30,260] [INFO] [logging] =======================================
[2024-09-29 14:16:30,261] [INFO] [logging]          New log starts here.          
[2024-09-29 14:16:30,262] [INFO] [logging] =======================================
[2024-09-29 14:16:30,263] [INFO] [logging] =======================================
[2024-09-29 14:16:30,264] [INFO] [logging] =======================================
[2024-09-29 14:16:30,266] [INFO] [rng] Rng seed: 1727612190266331


NameError: name 'make_body' is not defined

In [8]:


def simulate_manual_scene(
    scene: Scene,
    render_backend: RenderBackend = RenderBackend.GLFW,
) -> None:
    """
    Simulate a scene for checking if a robot was built correctly.

    :param scene: The scene to simulate.
    :param render_backend: The render backend.
    """
    logging.info("Simulating scene")

    model, mapping = scene_to_model(
        scene, STANDARD_SIMULATION_TIMESTEP, cast_shadows=False, fast_sim=False
    )
    data = mujoco.MjData(model)
    viewer = CustomMujocoViewer(
        model, data, mode=CustomMujocoViewerMode.MANUAL, backend=render_backend
    )

    # Compute forward dynamics without actually stepping forward in time.
    mujoco.mj_forward(model, data)

    control_interface = ControlInterfaceImpl(
        data=data,
        abstraction_to_mujoco_mapping=mapping,
    )

    """Here we set our values for cycling different positions."""
    prev_position: int = (
        0  # This is the initial idle position for all hinges (index of the positions).
    )
    positions: list[float] = [
        0.0,
        0.5,
        1.0,
        -0.5,
        -1.0,
    ]  # Those are the possible cycle positions we want.
    target, current = (
        0.0,
        0.0,
    )  # Here we can check whether we are currently on the correct position.
    try:
        while True:
            simulation_state = SimulationStateImpl(
                data=data, abstraction_to_mujoco_mapping=mapping, camera_views={}
            )
            scene.handler.handle(
                simulation_state, control_interface, STANDARD_CONTROL_FREQUENCY
            )
            # step simulation
            if target != current:
                """
                If we are not on the target we adjust our hinges accordingly.

                The stepsize of 0.0025 allows us to approximate the real robots movement better.
                If the simulated hinges are forced to the target location directly with big angles, the simulation detects instabilities and wont render, therefore we choose smaller steps.
                """
                step = 0.0025 if target > current else -0.0025
                for hinge in mapping.hinge_joint.keys():
                    control_interface.set_joint_hinge_position_target(
                        hinge.value, current + step
                    )
                current += step

            mujoco.mj_step(model, data)

            position = viewer.render()
            if position is not None and prev_position != position:
                prev_position = position
                target = positions[prev_position]

    except KeyboardInterrupt:
        """If we press ctrl-C this script will end with the finally clause."""
        pass
    finally:
        viewer.close_viewer()
        logging.info("Testing done.")


NameError: name 'RenderBackend' is not defined

In [14]:
%time
main()